# Exercises day 01

## Ex. 1

Implement a Linear Congruental Generator

In [5]:
def linear_congruental_generator(multiplier : int, shift : int, modulus: int, x0 : int = None) -> tuple[list[int], list[float]]:
    """
        Generates a list of random numbers using the linear congruental generator method.
        
        NOTE: random_nums and U will have length 'modulus' if the parameters follows the conditions
        of Theorem 1 Maximum Cycle Length.

        :param multiplier: The multiplier (a) used in the formula.
        :param shift: The shift (c) used in the formula.
        :param modulus: The modulus (M) used in the formula.
        :param x0: initial value (x0) used in the formula. If not provided, a random value will be used.
        
        :return random_nums: a list of randomly generated numbers.
        :return U: a list of random numbers between 0 and 1.
    """
    import numpy as np
    
    assert type(modulus) is int, "Modulus must be an integer."
    assert modulus > 0, "Modulus must be greater than 0."
    
    if x0 is None:
        x0 = np.random.randint(0, modulus)
    
    random_nums = [x0]
    U = [x0 / modulus]
    random_nums_generated = 0
    while random_nums_generated < modulus:
        random_nums.append((multiplier * random_nums[-1] + shift) % modulus) 
        U.append(random_nums[-1] / modulus)
        random_nums_generated += 1
    
    return random_nums, U
        
    
    
    
    

Testing whether the Linear Congruental Generator works as presented in the slides

In [6]:
M = 16
a = 5
c = 1
x0 = 3

ranom_nums, U = linear_congruental_generator(multiplier=a, shift=c, modulus=M, x0=x0)

print(f"Random numbers: \n{ranom_nums}\n")
print(f"U: \n{U}\n")

Random numbers: 
[3, 0, 1, 6, 15, 12, 13, 2, 11, 8, 9, 14, 7, 4, 5, 10, 3]

U: 
[0.1875, 0.0, 0.0625, 0.375, 0.9375, 0.75, 0.8125, 0.125, 0.6875, 0.5, 0.5625, 0.875, 0.4375, 0.25, 0.3125, 0.625, 0.1875]



### Run-tests

In [ ]:
def runs(random_nums : list[float]):
    """
        Calculates the runs of a list of random numbers.
        
        :param random_nums: a list of random numbers.
        
        :return R: a list of runs.
    """
    R = []
    
    previous_run = None
    run_length = 0
    for i in range(len(random_nums) - 1):
        if random_nums[i] < random_nums[i + 1]:
            if previous_run == "up" or previous_run is None:
                run_length += 1
            elif previous_run == "down":
                R.append(run_length)
                run_length = 1
                previous_run = "up"
        else:
            if previous_run == "down" or previous_run is None:
                run_length += 1
            elif previous_run == "up":
                R.append(run_length)
                run_length = 1
                previous_run = "down"
    
    return R
    

#### Above/Below


#### Up/Down

In [ ]:
def Up_Down(R : list[float], n : int = None) -> list[float]:
    """
        :param R: a list of random numbers between 0 and 1.
        :n: the amount of numbers in R. If not provided, the length of R will be used.
        
        :return Z: a list (of something I don't know yet).
    """
    if n is None:
        n = len(R)
        
    assert n == len(R), "'n' should be the amount of numbers in R."
    
    import numpy as np

    A = np.array([
        [4529.4, 9044.9, 13568, 18091, 22615, 27892],
        [9044.9, 18097, 27139, 36187, 45234, 55789],
        [13568, 27139, 40721, 54281, 67852, 83685],
        [18091, 36187, 54281, 72414, 90470, 111580],
        [22615, 45234, 67852, 90470, 113262, 139476],
        [27892, 55789, 83685, 111580, 139476, 172860]
        ])

    B = np.array([
        [1/6],
        [5/24],
        [11/120],
        [19/720],
        [29/5040],
        [1/840]
        ])
    
    Z = (1 / (n - 6)) * (R - n * B).T @ A @ (R - n * B)
    
    return Z

